In [ ]:
import subprocess
import pandas as pd
import re
from pathlib import Path

from pprint import pprint

In [ ]:
experiment_dir = Path("../data/gherkins/sample_data/test")
feature_file_dir = experiment_dir / "features"

gherkin_lint_path = Path("../gherkin-lint/.gherkin-lintrc")

In [ ]:
def run_lint(filepath):
    ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')

    result = subprocess.run(
    ["gherkin-lint", "-c", "../gherkin-lint/.gherkin-lintrc", filepath],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    shell=True
    )

    clean_output = ansi_escape.sub("", result.stdout)

    return clean_output

In [ ]:
reports = []

for file in feature_file_dir.rglob('*.feature'):
    filename = Path(file).name.replace('.feature', '')
    
    filepath = str(file)

    output = run_lint(filepath)
    
    if output:
        reports.append(output)

In [ ]:
len(reports)

In [ ]:
reports

In [ ]:
data = []

for report in reports:
    current_file = None

    # Parse the output line-by-line
    for line in report.splitlines():
        line = line.strip()
        
        # If it's a filename line
        if line.endswith(".feature"):
            current_file = line.split('\\')[-1]
        else:
            # Match: line_number [space] message [space] [space] rule
            match = re.match(r"^(\d+)\s+(.*?)\s{2,}(\S+)$", line)
            if match:
                line_number = int(match.group(1))
                description = match.group(2).strip()
                rule = match.group(3).strip()
                data.append({
                    "file": current_file,
                    "line": line_number,
                    "description": description,
                    "rule": rule
                })

    # Create the DataFrame
df = pd.DataFrame(data)
